In [1]:
!pip install streamlit duckduckgo_search transformers torch


In [2]:
app_code = '''
import streamlit as st
import requests
import time
from duckduckgo_search import DDGS
from transformers import pipeline

# Warnings Ignore
import warnings
warnings.filterwarnings('ignore')

# Streamlit Page Setup
st.set_page_config(page_title="Market Research & AI Use Case Generator", layout="wide")

# Load Summarizer and Generator ONCE
@st.cache_resource
def load_models():
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    generator = pipeline('text-generation', model='gpt2')
    return summarizer, generator

summarizer, generator = load_models()

# ----------------- Agents ----------------------

class ResearchAgent:
    def __init__(self, company):
        self.company = company

    def search_info(self):
        query = f"{self.company} industry overview"
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)
            texts = [r['body'] for r in results if 'body' in r]
            full_text = " ".join(texts)
            return full_text

    def summarize(self, text):
        if not text:
            return "No data found."
        summarized = summarizer(text, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
        return summarized

class UseCaseGeneratorAgent:
    def __init__(self, research_summary):
        self.research_summary = research_summary

    def generate_use_cases(self):
        prompt = f"""Given this research:

{self.research_summary}

Suggest 5 relevant AI/ML/GenAI use cases to improve operations, customer experience, or efficiency.
"""
        generated = generator(prompt, max_length=300, num_return_sequences=1)[0]['generated_text']
        lines = generated.split('\\n')
        use_cases = [line.strip("- ").strip() for line in lines if line.strip()]
        return use_cases[:5]

class DatasetCollectorAgent:
    def __init__(self, use_cases):
        self.use_cases = use_cases

    def search_datasets(self):
        datasets = {}
        for case in self.use_cases:
            query = f"{case} dataset site:kaggle.com OR site:huggingface.co"
            with DDGS() as ddgs:
                results = ddgs.text(query, max_results=2)
                links = [r['href'] for r in results if 'href' in r]
                datasets[case] = links
            time.sleep(1)
        return datasets

# ----------------- Streamlit App ----------------------

st.title("📊 Market Research & AI Use Case Generation Agent")

company_name = st.text_input("Enter Company or Industry Name:", "")

if st.button("Run Research"):
    if company_name:
        with st.spinner("Researching Industry..."):
            researcher = ResearchAgent(company_name)
            full_info = researcher.search_info()
            summary = researcher.summarize(full_info)

        st.subheader("🔎 Research Summary")
        st.write(summary)

        with st.spinner("Generating AI Use Cases..."):
            usecase_agent = UseCaseGeneratorAgent(summary)
            use_cases = usecase_agent.generate_use_cases()

        st.subheader("🚀 AI Use Cases Suggested")
        for idx, case in enumerate(use_cases, 1):
            st.write(f"{idx}. {case}")

        with st.spinner("Searching Resource Datasets..."):
            dataset_agent = DatasetCollectorAgent(use_cases)
            dataset_links = dataset_agent.search_datasets()

        st.subheader("📚 Dataset Resources")
        for uc, links in dataset_links.items():
            st.write(f"**{uc}**")
            for link in links:
                st.markdown(f"- [Dataset Link]({link})")
            st.markdown("---")

        st.success("✅ Process Completed!")

    else:
        st.warning("Please enter a valid company or industry name.")
'''

with open("app.py", "w") as f:
    f.write(app_code)


In [3]:
# Install required packages
!pip install streamlit pyngrok python-docx
!npm install -g localtunnel

import subprocess
import threading

# Function to run Streamlit
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.headless", "true", "--server.enableCORS", "false", "--server.enableXsrfProtection", "false"])

# Function to run localtunnel
def run_localtunnel():
    subprocess.run(["lt", "--port", "8501"])

# Start Streamlit in a thread
threading.Thread(target=run_streamlit).start()

# Start Localtunnel in another thread
threading.Thread(target=run_localtunnel).start()


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [4]:
!curl https://loca.lt/mytunnelpassword

34.23.1.177

In [5]:
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://olive-banks-pick.loca.lt
^C
